In [20]:
#import

In [21]:
class object:
    def __init__(self, x, y, z, name):
        self.x = x
        self.y = y
        self.z = z
        self._name = name #NOTE Must match the name assigned to the object in the simulation environment

    def name(self):
        return self._name


class action_module(object):
    '''
    '''

    def initial_module():
        string = r"{'action': 'MoveJ', 'value': {'joint1': 0.0, 'joint2': 0.0, 'joint3': 0.0, 'joint4': 0.0, 'joint5': 0.0, 'joint6': 0.0}, 'speed': 1.0}" #Don't need a test
        return string

    def initialize_ee():
        # Using MoveXYZ ensures that the end-effectors rotation remains constant
        # therefore, we must initilize the ee to be in the correct orientation (ready to grasp)
        #TODO: define the initial yaw, pitch, and roll values so that the ee is pointing down
        string = r"{'action': 'MoveJ', 'value': {'joint1': -58.10, 'joint2': 25.75, 'joint3': 27.17, 'joint4': -63.59, 'joint5': -71.41, 'joint6': 77.67}, 'speed': 1.0}"
        return string

    def pre_grasp_module(obj):
        # Pre-grasp (0.1 (z-axis) above the object)
        string = f"{{'action': 'MoveXYZ', 'value': {{'positionx': {obj.x}, 'positiony': {obj.y}, 'positionz': {obj.z+0.1}, 'speed': 1.0}}}}"
        return string

    def move_down_module():
        # Grasp (this sets the values to be inside the object?)
        string = r"{'action': 'MoveL', 'value': {'movex': 0.0, 'movey': 0.0, 'movez': -0.1}, 'speed': 1.0}"
        return string
    
    def attach_module(obj):
        # Attach #NOTE if this fails in the simulation, perhaps the name of the endeffector is wrong as this
        # assumes that all end_effectors are called EE_egp64
        string = f"{{'action': 'Attach', 'value': {{'object': '{obj.name()}', 'endeffector': 'EE_egp64'}}}}"
        return string
    
    def move_up_module():
        # We need to move up after grasping to avoid collision
        string = r"{'action': 'MoveL', 'value': {'movex': 0.00, 'movey': 0.0, 'movez': 0.1}, 'speed': 1.0}"
        return string
    
    def detach_module(obj):
        string = f"{{'action': 'Detach', 'value': {{'object': '{obj.name()}'}}}}"
        return string
    
    def close_module():
        return r"{'action': 'GripperClose'}" #Don't need a test
    
    def open_module():
        return r"{'action': 'GripperOpen'}" #Don't need a test
    
    def above_new_position(x,y,z):
        string = f"{{'action': 'MoveXYZ', 'value': {{'positionx': {x}, 'positiony': {y}, 'positionz': {z+0.1}, 'speed': 1.0}}}}"
        return string
    

In [22]:
# Create an instance of the Object class with coordinates (1, 1, 1)

# Testing
item = object(1, 2, 3, "item") # For now we can just manually set the location
x,y,z = 4,5,6
action = action_module
string1 = action.attach_module(item)


In [23]:
# Testing if they work correctly
#assert action.initialize_ee(action,item) == ?, "failure"
assert action.pre_grasp_module(item) == r"{'action': 'MoveXYZ', 'value': {'positionx': 1, 'positiony': 2, 'positionz': 3.1, 'speed': 1.0}}", "failure"
assert action.attach_module(item) == r"{'action': 'Attach', 'value': {'object': 'item', 'endeffector': 'EE_egp64'}}"
assert action.detach_module(item) == r"{'action': 'Detach', 'value': {'object': 'item'}}"
assert action.above_new_position(x,y,z) == r"{'action': 'MoveXYZ', 'value': {'positionx': 4, 'positiony': 5, 'positionz': 6.1, 'speed': 1.0}}", "failure"


In [24]:
# Example object:
box = object(1, 1, 0, "box")
# Target position
x1,y1,z1 = 1, 0, 0

# To generate the file:
model = action_module
action_list = []
#init
action_list.append(model.initial_module())
action_list.append(model.initialize_ee())
#pregrasp
action_list.append(model.pre_grasp_module(box))
#grasp
action_list.append(model.move_down_module())
#attach
action_list.append(model.attach_module(box))
#moveup
action_list.append(model.move_up_module())
#newlocation
action_list.append(model.above_new_position(x1,y1,z1))
action_list.append(model.move_down_module())
#detach
action_list.append(model.detach_module(box))
#init
action_list.append(model.move_up_module())
action_list.append(model.initial_module())

action_list
# Name of the text file
filename = 'list_elements.txt'

# Open the file in write mode
with open(filename, 'w') as file:
    # Iterate over the list and write each element to a separate line in the file
    for line in action_list:
        file.write(line + '\n')

In [25]:
# Example instruction
example_instructions = {'what': 'cup',
    'how': 'move',
    'location': {'precise': 'False',
     'relation': 'box',
     'direction': 'forward',
     'distance': '10 cm'}
}

In [26]:
# Based on the above code and instructions, let's generate instructions

def generate_instruction_set(instructions, save_file):
    target_object = instructions['what']
    task = instructions['how']
    location = instructions['location']
    match task:
        case 'move':
            # Currently we only go here
            if location['precise'] == 'True':
                # Precise location logic here:
                ""
            else: # Not precise:
                ""
        case 'some other action':
            print("")# This is a place-holder for future tasks

    return ""

def move_with_respect_to(relation, direction, distance):
    '''
        parameter relation: type object
        parameter direction: type string
        parameter action_module: type action_module

        return: String

        This function is basically for generating the target location of a object being moved
        and returning the string that corresponds that action
    '''
    # Push the salt shaker forward 5 cm from the pepper
    # Move the cup forward 10 cm relative to the box

    am = action_module

    match direction:
        case 'forward':

            string = am.above_new_position(x,y,z)
        case 'beside':
            ""
        case 'toward':
            ""
        case 'right':
            ""
        case 'left':
            ""
        case 'closer':
            ""
        case 'behind':
            ""
        

In [27]:
generate_instruction_set(example_instructions, "place_holder")

''